# Uso de modelos de embeddings de OpenAI

## Instalación y carga de librerías

In [ ]:
!pip install openai
!pip install tiktoken

In [2]:
import openai
from getpass import getpass
import os
import pandas as pd

In [ ]:
OPENAI_API_KEY = getpass('Enter the secret value: ')
openai.api_key = OPENAI_API_KEY

Enter the secret value: ··········


## Cargar dataset

In [1]:
import subprocess

# Ejecutar el comando desde Python
subprocess.call(["gsutil", "-m", "cp", "-r", "gs://zaia-health-dev_pocs/etl/typeSideEffect", "C:\Developer_1.0\Proyecto_GCP\\"], shell=True)

<>:4: SyntaxWarning: invalid escape sequence '\D'
<>:4: SyntaxWarning: invalid escape sequence '\D'
C:\Users\JORGE-PC\AppData\Local\Temp\ipykernel_4652\17150008.py:4: SyntaxWarning: invalid escape sequence '\D'
  subprocess.call(["gsutil", "-m", "cp", "-r", "gs://zaia-health-dev_pocs/etl/typeSideEffect", "C:\Developer_1.0\Proyecto_GCP\\"], shell=True)


0

In [8]:
import pandas as pd

df = pd.read_csv('C:\\Developer_1.0\\Proyecto_GCP\\typeSideEffect\\2024_03_07_1709771913793_0.csv')

In [9]:
df.head()

,_airbyte_ab_id,_airbyte_emitted_at,_ab_cdc_deleted_at,_ab_cdc_lsn,_ab_cdc_updated_at,comment,groupSideEffect,value
0,9549724d-2608-483d-84ab-d5315ecb7008,1709771914974,NaN,NaN,NaN,"{""en"": ""Thinning hair"", ""es"": ""Adelgazamiento ...",general,thinninghair
1,52fb3369-cf94-49c9-bfb9-362c51f3785b,1709771914974,NaN,NaN,NaN,"{""en"": ""Low red blood cell count"", ""es"": ""Recu...",general,lowwhitebloodcellcount
2,d17852a0-29b8-4edb-96ce-5ed3645556a8,1709771914974,NaN,NaN,NaN,"{""en"": ""Muscle pain"", ""es"": ""Dolor muscular""}",general,musclepain
3,9454e58d-d5d7-4f70-8951-98b192d8cca8,1709771914974,NaN,NaN,NaN,"{""en"": ""Confusion"", ""es"": ""Confusión""}",general,confusion
4,2639a983-4b45-4a90-ad34-72c8208262e8,1709771914974,NaN,NaN,NaN,"{""en"": ""Painful swelling in legs"", ""es"": ""Hinc...",general,painfullegs


In [10]:
import pandas as pd

# Divide la columna "comment" por coma y expandela en dos nuevas columnas
nuevos_comentarios = df['comment'].str.split(',', expand=True)

# Añade las nuevas columnas al DataFrame (opcional)
df = pd.concat([df, nuevos_comentarios], axis=1)

#elimina colummnas que se crearon de mas
df = df.drop([2,3,4,5,6,7,8,9,10],axis=1)

#renombra las 2 columnas nuevas
df = df.rename(columns={0: 'ingles', 1: 'español'})

#modifica la columna ingles y la normaliza
df['ingles'] = df['ingles'].str.strip('{')
df['ingles'] = df['ingles'].str.replace('["]', '', regex=True)
df['ingles'] = df['ingles'].str[4:]

#modifica la columna español y la normaliza
df['español'] = df['español'].str.strip('}')
df['español'] = df['español'].str.replace('["]', '', regex=True)
df['español'] = df['español'].str[5:]

#elimina la columna vieja
df = df.drop('comment', axis=1)

# muestra imprime
df.head()





,_airbyte_ab_id,_airbyte_emitted_at,_ab_cdc_deleted_at,_ab_cdc_lsn,_ab_cdc_updated_at,groupSideEffect,value,ingles,español
0,9549724d-2608-483d-84ab-d5315ecb7008,1709771914974,NaN,NaN,NaN,general,thinninghair,Thinning hair,Adelgazamiento del cabello
1,52fb3369-cf94-49c9-bfb9-362c51f3785b,1709771914974,NaN,NaN,NaN,general,lowwhitebloodcellcount,Low red blood cell count,Recuento bajo de glóbulos blancos
2,d17852a0-29b8-4edb-96ce-5ed3645556a8,1709771914974,NaN,NaN,NaN,general,musclepain,Muscle pain,Dolor muscular
3,9454e58d-d5d7-4f70-8951-98b192d8cca8,1709771914974,NaN,NaN,NaN,general,confusion,Confusion,Confusión
4,2639a983-4b45-4a90-ad34-72c8208262e8,1709771914974,NaN,NaN,NaN,general,painfullegs,Painful swelling in legs,Hinchazón dolorosa en piernas


## Evaluar cantidad de tokens a procesar

In [ ]:
import tiktoken

def num_tokens_from_string(text, encodig_name):
    encoding = tiktoken.get_encoding(encodig_name)
    num_tokens = len(encoding.encode(text))
    return num_tokens


In [ ]:
df['total_tokens'] = df['FOOD NAME'].apply(lambda x : num_tokens_from_string(x,'cl100k_base') )

In [ ]:
sum(df['total_tokens'])

2947

## Generando emebeddings

In [ ]:
from openai import OpenAI
client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

df['ada_embedding'] = df.combined.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
df.to_csv('output/embedded_1k_reviews.csv', index=False)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
def get_embedding(text, model='text-embedding-ada-002'):
    text = text.replace('\n','')
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [ ]:
df['ada_embeddings'] =  df['FOOD NAME'].apply(lambda x :get_embedding(x, model='text-embedding-ada-002') )

APIRemovedInV1: 

You tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
df.head()

,FOOD NAME,SCIENTIFIC NAME,GROUP,SUB GROUP,total_tokens,ada_embeddings
0,Angelica,Angelica keiskei,Herbs and Spices,Herbs,2,"[0.0061722793616354465, -0.010086163878440857,..."
1,Savoy cabbage,Brassica oleracea var. sabauda,Vegetables,Cabbages,4,"[0.005525844171643257, -0.004996671807020903, ..."
2,Silver linden,Tilia argentea,Herbs and Spices,Herbs,3,"[-0.004518164321780205, 0.020059844478964806, ..."
3,Kiwi,Actinidia chinensis,Fruits,Tropical fruits,2,"[-0.004589142743498087, -0.010032080113887787,..."
4,Allium (Onion),Allium,Vegetables,Onion-family vegetables,6,"[0.013222076930105686, -0.019921524450182915, ..."


In [ ]:
embedding_prueba = get_embedding('esto es una prueba de embeddings para openAI')

APIRemovedInV1: 

You tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
len(embedding_prueba)

1536

## Creando datasets para visualizar

In [ ]:
df_embeddings = pd.DataFrame(list(df['ada_embeddings']))

In [ ]:
df_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,0.006172,-0.010086,0.004700,-0.028833,-0.005396,0.013111,-0.016375,0.007222,-0.009960,-0.028140,...,0.024303,-0.011538,0.004470,-0.022611,-0.008880,0.030938,0.008394,-0.003977,-0.022477,-0.015109
1,0.005526,-0.004997,0.014958,-0.010917,-0.002466,0.021154,-0.008922,-0.024310,-0.018050,-0.039742,...,0.017947,-0.019807,0.026837,-0.032636,-0.019397,0.020012,-0.025593,0.010494,-0.007736,0.000720
2,-0.004518,0.020060,-0.002007,-0.024115,-0.016690,0.020476,-0.041274,-0.001940,-0.001276,-0.035474,...,0.002878,-0.022356,0.038562,-0.026344,-0.032439,0.021174,-0.000270,-0.020436,-0.011561,-0.033460
3,-0.004589,-0.010032,-0.009249,-0.010006,-0.007794,0.023091,-0.014398,-0.016183,-0.001687,-0.036532,...,0.030944,-0.012121,0.022496,-0.011804,-0.005365,0.010601,-0.021759,-0.001688,-0.014049,-0.020672
4,0.013222,-0.019922,0.002163,-0.010105,-0.018612,0.022279,-0.018114,-0.015285,-0.008900,-0.020511,...,0.020210,-0.034447,0.031513,-0.010989,-0.019371,0.000302,0.010269,0.001256,-0.001141,0.012417


In [ ]:
df_embeddings.to_csv('embedding_food.tsv',sep='\t',index=False, header=False)

In [ ]:
df[['FOOD NAME','GROUP','SUB GROUP']].to_csv('labels_food.tsv',sep='\t',index=False, header=True)